In [20]:
library(lme4)
library(lmerTest)
library(stargazer)
library(MatchIt)
library("lmtest") #coeftest
library("sandwich") #vcovCL

Loading required package: Matrix

Warning message:
“package ‘lmerTest’ was built under R version 3.6.3”

Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step


Warning message:
“package ‘stargazer’ was built under R version 3.6.3”

Please cite as: 


 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 


Warning message:
“package ‘lmtest’ was built under R version 3.6.3”
Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Warning message:
“package ‘sandwich’ was built under R version 3.6.3”


In [21]:
entertainment_topics <- c(1,10,19,24,44)
promotion_topics <- c(3,16,31)
politics_topics <- c(11,23,30,41,46,47)
sports_topics <- c(48)
topics_of_interest <- c(1,10,19,24,44,3,16,31,11,23,30,41,46,47,48)

In [22]:
topic_list <- list(entertainment=entertainment_topics,
                   promotion=promotion_topics,
                   politics=politics_topics,
                   sports=sports_topics)

In [25]:
country.pair.dirs <- list.dirs(path="/shared/2/projects/cross-lingual-exchange/data/multilingual_friend_effect_dataframes_top100")
country.pair.dirs <- country.pair.dirs[2:length(country.pair.dirs)]
all.data.frames <- vector(mode="list",length=2*length(country.pair.dirs))

In [ ]:
for (topic_id in 1:1){
    broad_topic_name <- names(topic_list)[[topic_id]]
    narrow_topics <- topic_list[[topic_id]]
    out.file <- paste("/shared/2/projects/cross-lingual-exchange/results/",
                      broad_topic_name,
                      "_att_temporal_hashtag_top100_with_rt.tsv",sep='')
    x <- data.frame('Country Pair','Country','Estimate', 'Std. Error', 'P-value', 'Robust Std. Error', 'Robust P-value')
    write.table(x, file = out.file, sep = "\t", append = TRUE, quote = FALSE, col.names = FALSE, row.names = FALSE)
    for(i in 4:4)){
        country.pair.dir <- country.pair.dirs[[i]]
        country.pair <- unlist(strsplit(basename(country.pair.dir),'_'))
        for(j in 1:1){
            country <- country.pair[[j]]
            filename <- paste(country.pair.dir,"/",country.pair[[j]],".gz",sep='')
            df <- read.table(gzfile(filename),header=TRUE,sep='\t')
            ix <- as.integer(2*i + 1 - j)
            df <- df[df$monolingual_majority == 1,]
            df$id_str <- NULL
            df$country <- NULL
            df <- df[!rowSums(df < 0), ]
            df$log_degree <- log(df$num_neighbors + 1)
            df$log_friends <- log(df$friends_count + 1)
            df$log_followers <- log(df$followers_count + 1)
            df$log_statuses <- log(df$statuses_count + 1)
            df$log_tweets <- log(df$decahose_tweet_count + 1)
            df$log_favourites <- log(df$favourites_count + 1)
            df$scaled_betw <- 1000000 * df$betw
            df$log_betw <- log(df$scaled_betw + 1)
            df$log_rate <- log(df$post_rate + 1)
            df$log_age <- log(df$account_age + 1) # make that linear
            df$outcome <- 0
            for(narrow_topic in narrow_topics){
              df$outcome <- df$outcome + 
                df[[paste("has_temporal_hashtags_top100_other_language_topic_",narrow_topic,sep='')]]
            }
            cat(head(df$outcome,'\n'))
            if(topic_id == 1){
                all.data.frames[[ix]] <- df
            }

            if (sum(df$has_bilingual_neighbor) >= 100 && sum(df$outcome)>=100){
                m <- matchit(has_bilingual_neighbor ~ log_degree
                           + verified + log_friends + log_followers + log_statuses + log_favourites
                           + log_age + log_rate + log_tweets,
                           data=df,method='subclass',subclass=25,
                           estimand='ATT')
                m.data <- match.data(m)
                fit <- glm(outcome ~ has_bilingual_neighbor + log_degree
                           + verified + log_friends + log_followers + log_statuses + log_favourites
                           + log_age + log_rate + log_tweets
                           ,data=m.data,weights=weights,
                         family=quasibinomial(link="logit"),
                          )
                rr <- as.numeric(exp(coef(fit)[2]))
                se <- as.numeric(summary(fit)$coefficients[, 2][2])
                p <- as.numeric(summary(fit)$coefficients[,4][2])
                res.robust <- coeftest(fit,vcov. = vcovHC)
                se.robust <- res.robust[2,2]
                p.robust <- res.robust[2,4]
                x <- c(paste(country.pair[1],country.pair[2],sep='_'),country,rr,se,p,se.robust,p.robust)
                cat(x,'\n')
                x <- as.matrix(t(x))
                write.table(x, file = out.file, sep = "\t", append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)
            }
        }
    }     
}

In [28]:
m.data

ERROR: Error in eval(expr, envir, enclos): object 'm.data' not found


In [144]:
df <- do.call("rbind", all.data.frames)

In [ ]:
out.file <- "/shared/2/projects/cross-lingual-exchange/results/overall_att_temporal_hashtag_top100_with_rt_by_topic.tsv"
x <- data.frame('Topic','Estimate', 'Std. Error', 'P-value', 'Robust Std. Error', 'Robust P-value')
write.table(x, file = out.file, sep = "\t", append = TRUE, quote = FALSE, col.names = FALSE, row.names = FALSE)
    

In [145]:
for (topic_of_interest in topics_of_interest){
    df$outcome <- df[[paste("has_temporal_hashtags_top100_other_language_topic_",topic_of_interest,sep='')]]
    m <- matchit(has_bilingual_neighbor ~ log_degree
               + verified + log_friends + log_followers + log_statuses + log_favourites
               + log_age + log_rate + log_tweets,
               data=df,method='subclass',subclass=25,
               estimand='ATT')
    m.data <- match.data(m)
    fit <- glm(outcome ~ has_bilingual_neighbor + log_degree
               + verified + log_friends + log_followers + log_statuses + log_favourites
               + log_age + log_rate + log_tweets
               ,data=m.data,weights=weights,
             family=quasibinomial(link="logit"),
              )
    rr <- as.numeric(exp(coef(fit)[2]))
    se <- as.numeric(summary(fit)$coefficients[, 2][2])
    p <- as.numeric(summary(fit)$coefficients[,4][2])
    res.robust <- coeftest(fit,vcov. = vcovHC)
    se.robust <- res.robust[2,2]
    p.robust <- res.robust[2,4]
    x <- c(topic_of_interest,rr,se,p,se.robust,p.robust)
    cat(x,'\n')
    x <- as.matrix(t(x))
    write.table(x, file = out.file, sep = "\t", append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

all all 15.56989328832 0.00300388217021281 0 0.0037875372125615 0 
